In [61]:
import os

In [62]:
%pwd

'C:\\Users\\Surya Teja\\OneDrive\\Desktop\\text-summarizer-project\\Text-Summarizer-Project'

In [63]:
os.chdir(r"C:\Users\Surya Teja\OneDrive\Desktop\text-summarizer-project\Text-Summarizer-Project")

In [64]:
import sys
sys.path.append("../")
print(f"Added {os.getcwd()} to Python path")

Added C:\Users\Surya Teja\OneDrive\Desktop\text-summarizer-project\Text-Summarizer-Project to Python path


In [48]:
!pip install pyyaml python-box ensure

In [49]:
with open('src/textSummarizer/utils/common.py', 'r') as file:
    content = file.read()

content = content.replace('content = yaml.safe_load(file)', 'content = yaml.safe_load(yaml_file)')

with open('src/textSummarizer/utils/common.py', 'w') as file:
    file.write(content)

print("File updated successfully")

File updated successfully


In [51]:
# First, read the content of common.py
with open('src/textSummarizer/utils/common.py', 'w') as file:
    file.write('''import os
from box.exceptions import BoxValueError
import yaml
from src.textSummarizer.logging import logger
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any

@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml, "r") as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories
    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"created directory at: {path}")

@ensure_annotations
def get_size(path: Path) -> str:
    """get size in KB
    Args:
        path (Path): path of the file
    Returns:
        str: size in KB
    """
    size_in_kb = round(os.path.getsize(path)/1024)
    return f"~ {size_in_kb} KB"
''')

In [60]:
# Verify the content of common.py
with open('src/textSummarizer/utils/common.py', 'r') as file:
    print(file.read())

import os
from box.exceptions import BoxValueError
import yaml
from src.textSummarizer.logging import logger
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any

@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml, "r") as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories
    Args:
        path_to_director

In [65]:
%pwd

'C:\\Users\\Surya Teja\\OneDrive\\Desktop\\text-summarizer-project\\Text-Summarizer-Project'

In [66]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [67]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [68]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [69]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [70]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [73]:
from box import ConfigBox
from pathlib import Path
import yaml

# Let's create our own read_yaml function
def my_read_yaml(path_to_yaml: Path) -> ConfigBox:
    try:
        with open(path_to_yaml, 'r') as f:
            content = yaml.safe_load(f)
            return ConfigBox(content)
    except Exception as e:
        raise e

# Now let's use this function in ConfigurationManager
ConfigurationManager.__init__ = lambda self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH: (
    setattr(self, 'config', my_read_yaml(config_filepath)),
    setattr(self, 'params', my_read_yaml(params_filepath)),
    create_directories([my_read_yaml(config_filepath).artifacts_root]),
    None)[-1]

In [74]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-02 13:56:45,950] [INFO] [textSummarizerLogger] - created directory at: artifacts
[2025-11-02 13:56:45,951] [INFO] [textSummarizerLogger] - created directory at: artifacts/data_ingestion
[2025-11-02 13:56:45,951] [INFO] [textSummarizerLogger] - created directory at: artifacts/data_ingestion
[2025-11-02 13:56:49,009] [INFO] [textSummarizerLogger] - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4367:CFE2A:4E31B7:85E5FE:690715C8
Accept-Ranges: bytes
Date: Sun, 02 Nov 2025 08:26:48 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10221-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-T